In [2]:
import pandas as pd

In [3]:
import breakdown # remarkを要素に分割/flag
import embedding # 文章をembedding
from embedding import get_embedding, cos_sim

In [ ]:
import pandas as pd 

def prepare_dataframe(csv):
    df = pd.read_csv(csv)
    df["embedded"] = df["embedded"].map(lambda x: eval(x))
    return df

In [5]:
# dataframeの設定
df = prepare_dataframe("new_new_embedded_all.csv")

In [9]:
class Sentence:
    def __init__(self, verb, agent, role, team):
        self.verb = verb
        self.agent = agent
        self.role = role
        self.team = team

In [48]:
class Comment:
    def __init__(self, remark, talker):
        self.remark = remark
        self.talker = talker
        self.flag = None 
        
    def remark_to_protocol(self):
        """
        大元のメソッド. Commentのインスタンスに対して呼び出すと、remarkがプロトコルに変換され、flagが立つ
        """
        vectorlist = self.remark_to_vectors(self.talker, self.remark)
        return self.vectors_to_protocols(vectorlist)

    def remark_to_vectors(self, talker, remark):
        """
        gptの処理を呼び出して、remarkを要素に分割し、各要素をembeddingしたリストを返す
        flagも設定する
        """
        return_from_gpt = breakdown.get_list(talker, remark)
        sentences = return_from_gpt[0]
        self.flag = return_from_gpt[1]

        # 各要素をembeddingしたリスト
        return [get_embedding(sentence) for sentence in sentences]
    
    def ruizido(self, vector):
        """
        入力文の要素の一つをembedしたベクトルと、データフレームのembedded列のベクトルとの類似度を計算し、最も類似度が高い文のindexを返す
        """
        similarity_list = [] #一旦、類似度だけを格納しておくためのリスト
        for i in df["embedded"]:
            similarity_list.append(cos_sim(i, vector))
            
        index = similarity_list.index(max(similarity_list))
        max_sim = max(similarity_list)
        
        
        #最大の類似度と、その文のindexを返す
        return (max_sim, index)

    #最終的なリストの作成
    #from send import Sentence

    def vectors_to_protocols(self, vectorlist): #vectorlistはto_vectors(talker, comment)の返り値
        protocol_list = []
        for i in vectorlist:
            verb = df.loc[self.ruizido(i)[1], "Verb"]
            agent = df.loc[self.ruizido(i)[1], "Agent"]
            role = df.loc[self.ruizido(i)[1], "Role"]
            team = df.loc[self.ruizido(i)[1], "Team"]
            
            
            
            sentence = Sentence(verb, agent, role, team)
            protocol_list.append(sentence)
        
        return protocol_list
        

In [49]:
comment1 = Comment("Agent[02]を占ったら黒だったよ。Agent[01]を占ったら白だったよ。", "Agent[00]") 
protocol_list = comment1.remark_to_protocol()

In [50]:
for protocol in protocol_list:
    print(protocol.verb, protocol.agent, protocol.role, protocol.team)
print(comment1.flag)

DIVINED Agent[02] nan BLACK
DIVINED Agent[01] nan WHITE
0


In [47]:
comment2 = Comment("Agent[00]の言っていることは信じられない。今夜はAgent[00]に投票するよ。", "Agent[01]") 
protocol_list = comment1.remark_to_protocol()
for protocol in protocol_list:
    print(protocol.verb, protocol.agent, protocol.role, protocol.team)
print(comment1.flag)

NOT SUSPECT Agent[00] nan nan
VOTE Agent[00] nan nan
0
